In [ ]:
import os
import cv2
import json
from datetime import datetime
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [ ]:
# Function 1: Extract GPS metadata

def get_gps_from_image(image_path):
    img = Image.open(image_path)
    exif_data = img._getexif()
    if not exif_data:
        return None

    gps_info = {}
    for tag, value in exif_data.items():
        decoded = TAGS.get(tag, tag)
        if decoded == "GPSInfo":
            for t in value:
                gps_tag = GPSTAGS.get(t, t)
                gps_info[gps_tag] = value[t]

    if not gps_info:
        return None

    def convert_to_degrees(value):
        d, m, s = value
        return d[0]/d[1] + (m[0]/m[1])/60 + (s[0]/s[1])/3600

    lat = convert_to_degrees(gps_info['GPSLatitude'])
    lon = convert_to_degrees(gps_info['GPSLongitude'])
    if gps_info['GPSLatitudeRef'] != 'N':
        lat = -lat
    if gps_info['GPSLongitudeRef'] != 'E':
        lon = -lon

    return {"latitude": lat, "longitude": lon}


In [ ]:
# Function 2: Parse YOLO label file
def parse_yolo_output(txt_path, image_path, class_names):
    detections = []
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not read image: {image_path}")
        return detections
    h, w = img.shape[:2]

    with open(txt_path, 'r') as f:
        for line in f.readlines():
            values = line.strip().split()
            if len(values) < 5:
                continue

            cls, x_c, y_c, bw, bh = map(float, values[:5])
            cls = int(cls)

            # ensure class ID is valid
            if cls < 0 or cls >= len(class_names):
                print(f"Invalid class ID {cls} in {txt_path}, skipping")
                continue

            cls_name = class_names[cls]
            x_center, y_center = int(x_c * w), int(y_c * h)
            box_w, box_h = int(bw * w), int(bh * h)
            x1, y1 = x_center - box_w // 2, y_center - box_h // 2
            x2, y2 = x_center + box_w // 2, y_center + box_h // 2

            detections.append({
                "class": cls_name,
                "bbox": [x1, y1, x2, y2]
            })
    return detections

In [ ]:
# Function 3: Combine GPS + YOLO into JSON

def create_detection_json(image_path, txt_path, class_names):
    gps_data = get_gps_from_image(image_path)
    detections = parse_yolo_output(txt_path, image_path, class_names)

    final_output = {
        "image_path": image_path,
        "timestamp": datetime.utcnow().isoformat(),
        "location": gps_data or {"latitude": None, "longitude": None},
        "detections": detections
    }
    return final_output

In [ ]:
# Main Script

if __name__ == "__main__":
    image_folder = r"C:\Users\Mahika\Desktop\cv_gai\data\test\images"
    label_folder = r"C:\Users\Mahika\Desktop\cv_gai\data\test\labels"
    output_json_path = r"C:\Users\Mahika\Desktop\cv_gai\detections_with_gps.json"

    class_names = ['not-pothole', 'potholes']

    results = []
    for img_file in os.listdir(image_folder):
        if img_file.endswith((".jpg", ".png", ".jpeg")):
            image_path = os.path.join(image_folder, img_file)
            txt_path = os.path.join(label_folder, os.path.splitext(img_file)[0] + ".txt")

            if os.path.exists(txt_path):
                detection_entry = create_detection_json(image_path, txt_path, class_names)
                results.append(detection_entry)
            else:
                print(f"Label not found for {img_file}")

    # save combined JSON file
    with open(output_json_path, "w") as f:
        json.dump(results, f, indent=4)

    print(f"Detection results with GPS saved to {output_json_path}")


✅ Detection results with GPS saved to C:\Users\Mahika\Desktop\cv_gai\detections_with_gps.json
